In [177]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [173]:
import os
import shutil
import sys
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
import tensorflow as tf
import bottleneck as bn
import keras
import math

In [174]:
import argparse
import time
import csv
import pickle
import operator
import datetime
import os

In [178]:
cd /content/drive/MyDrive/Colab Notebooks/recommendation_system/deep_learning/session_based_rec/test/datasets/HandM

/content/drive/MyDrive/Colab Notebooks/recommendation_system/deep_learning/session_based_rec/test/datasets/HandM


In [179]:
tra=pickle.load(open('HandM/0506tra.txt', 'rb'))
tes=pickle.load(open('HandM/0506tes.txt', 'rb'))

In [180]:
def max_len_check(all_usr_pois):

    us_lens = [len(upois) for upois in all_usr_pois] # 이게 오래 걸리는듯.
    #print("us_lens in data_masks")
    #print(us_lens,'\n')
    
    len_max = max(us_lens)
    return  len_max

def split_validation(train_set, valid_portion):
    train_set_x, train_set_y = train_set
    n_samples = len(train_set_x)
    sidx = np.arange(n_samples, dtype='int32')
    np.random.shuffle(sidx)
    n_train = int(np.round(n_samples * (1. - valid_portion)))
    valid_set_x = [train_set_x[s] for s in sidx[n_train:]]
    valid_set_y = [train_set_y[s] for s in sidx[n_train:]]
    train_set_x = [train_set_x[s] for s in sidx[:n_train]]
    train_set_y = [train_set_y[s] for s in sidx[:n_train]]

    return (train_set_x, train_set_y), (valid_set_x, valid_set_y)


class Data():
    def __init__(self, data, one_day_max=100,shuffle=False, graph=None):
        inputs = data[0]
        len_max = max_len_check(inputs)
        self.inputs = inputs
        
        self.len_max = len_max
        self.targets = np.asarray(data[1])
        self.length = len(inputs)
        self.shuffle = shuffle
        self.graph = graph
        self.one_day_max = one_day_max

    def generate_batch(self, batch_size):
        if self.shuffle:
            shuffled_arg = np.arange(self.length)
            np.random.shuffle(shuffled_arg)
            self.inputs = self.inputs[shuffled_arg]
            #self.mask = self.mask[shuffled_arg]
            self.targets = self.targets[shuffled_arg]
        n_batch = int(self.length / batch_size)
        if self.length % batch_size != 0:
            n_batch += 1
        slices = np.split(np.arange(n_batch * batch_size), n_batch)
        slices[-1] = slices[-1][:(self.length - batch_size * (n_batch - 1))]
        return slices

    def get_slice(self, i):
        #inputs, mask, targets = self.inputs[i], self.mask[i], self.targets[i]
        inputs = []
        for jk in i :
          inputs.append(self.inputs[jk])

        targets = self.targets[i]

        us_lens = [len(upois) for upois in inputs] # 이게 오래 걸리는듯.
        
        
        inputs = [[j + [0] * (self.one_day_max - len(j)) for j in one_seq] for one_seq in inputs]
        
        
        for idx,le in enumerate(us_lens):
          for jj in range(self.len_max - le):
            inputs[idx].append(self.one_day_max * [0])
        

        
        
        items, n_node, A, alias_inputs = [], [], [], []
        for u_input in inputs:
            n_node.append(len(np.unique(u_input))) 
        max_n_node = np.max(n_node) # 해당 slices 중 한 sequence 안에서 가질 수 있는 아이템의 종류의 최댓값
        # 0 패딩 후에 개수를 따지는 것이기 때문에 id 개수 +1 임.
        
        
        
        for u_input in inputs:
            node = np.unique(u_input)
            
           
            items.append(node.tolist() + (max_n_node - len(node)) * [0])
            
            u_A = np.zeros((max_n_node, max_n_node)) # 균일한 크기로 생성.
            
            
        

            for i in range(len(u_input)):
              # same time point loop
              length=len(u_input[i])
              for j in range(length):
                for k in range(length):
                  if u_input[i][k] == 0 or  u_input[i][j] == 0:
                    break
                  u = np.where(node == u_input[i][j])[0][0]
                  v = np.where(node == u_input[i][k])[0][0]
                  u_A[u][v] +=2
            
            for i in range(len(u_input)):
              for kk in range(len(u_input[i])-1):
                jj=u_input[i][kk]
                ll=u_input[i][kk+1]
                if jj == 0 or ll == 0:
                  break
                u = np.where(node == jj)[0][0]
                v = np.where(node == ll)[0][0]
                u_A[u][v] +=1
              
            
        
            u_sum_in = np.sum(u_A, 0) # degree scaling
            u_sum_in[np.where(u_sum_in == 0)] = 1
            u_A_in = np.divide(u_A, u_sum_in)
            u_sum_out = np.sum(u_A, 1)
            u_sum_out[np.where(u_sum_out == 0)] = 1
            u_A_out = np.divide(u_A.transpose(), u_sum_out)
            u_A = np.concatenate([u_A_in, u_A_out]).transpose()
            A.append(u_A)
            u_alias = []
            for aday in u_input:
              seq_aday = []
              for element in aday:
                seq_aday.append(np.where(node == element)[0][0])
              u_alias.append(seq_aday)
            alias_inputs.append(u_alias)

        return alias_inputs, A, items, targets


In [181]:
train_data = tra
test_data = tes

In [182]:
train_data1 = Data(train_data,one_day_max=79,shuffle=False) # 같은 날짜 0 padding만 하는 것은 6분 정도.
test_data1 = Data(test_data,one_day_max=85,shuffle=False) # 같은 날짜 0 padding만 하는 것은 6분 정도.

In [183]:
class opt:
  def __init__(self,step,hidden_size,batch_size,nonhybrid=None):
    self.step=step
    self.hiddenSize = hidden_size
    #self.n_node = n_node
    self.batchSize = batch_size
    self.nonhybrid = nonhybrid
    self.lr = 0.001
    self.l2 = 1e-5
    self.lr_dc_step = 3
    self.lr_dc = 0.1
    self.epoch = 30
    self.patience = 10
oopt=opt(step = 1,hidden_size=100,batch_size=32)
n_node = 43849

In [184]:
class GNN(tf.keras.layers.Layer):
    def __init__(self, hidden_size, step=1):
        super(GNN, self).__init__()
        self.step = step
        self.hidden_size = hidden_size
        self.input_size = hidden_size * 2
        self.gate_size = 3 * hidden_size
        # self.w_ih = Parameter(torch.Tensor(self.gate_size, self.input_size))
        self.stdv = 1/math.sqrt(self.hidden_size)
        self.w_ih = self.add_weight(shape=(self.input_size,self.gate_size),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),name='w_ih')
        '''
        AttributeError: module 'keras.api._v2.keras.initializers' has no attribute 'Uniform'
        add_weight doesn't have the attribute named 'kernel_initializer'
        '''
        # self.w_hh = Parameter(torch.Tensor(self.gate_size, self.hidden_size))
        self.w_hh = self.add_weight(shape=(self.hidden_size,self.gate_size),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),name = 'w_hh')
        # self.b_ih = Parameter(torch.Tensor(self.gate_size))
        self.b_ih = self.add_weight(shape=(self.gate_size,),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),name = 'b_ih')
        
        # self.b_hh = Parameter(torch.Tensor(self.gate_size))
        self.b_hh = self.add_weight(shape=(self.gate_size,),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),name = 'b_hh')
                
        # self.b_iah = Parameter(torch.Tensor(self.hidden_size))
        self.b_iah = self.add_weight(shape=(self.hidden_size,),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),name = 'b_iah')
        # self.b_oah = Parameter(torch.Tensor(self.hidden_size))
        self.b_oah = self.add_weight(shape=(self.hidden_size,),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),name = 'b_oah')

        # self.linear_edge_in = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_edge_in = tf.keras.layers.Dense(self.hidden_size, 
                                                    use_bias=True,
                               kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                               bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        '''
        TypeError: ('Unknown keyword argument:', 'bias') - > use_bias=True in Dense
        '''
        # self.linear_edge_out = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_edge_out = tf.keras.layers.Dense(self.hidden_size,
                                    use_bias=True ,
                               kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                               bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        # self.linear_edge_f = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_edge_f = tf.keras.layers.Dense(self.hidden_size,
                                    use_bias=True ,
                               kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                               bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))

    def Cell(self, A, hidden):
        input_in = tf.matmul(A[:, :, :A.shape[1]], self.linear_edge_in(hidden)) + self.b_iah
        input_out = tf.matmul(A[:, :, A.shape[1]: 2 * A.shape[1]],self.linear_edge_out(hidden)) + self.b_oah
        inputs = tf.concat([input_in, input_out], 2) 
        gi = tf.matmul(inputs, self.w_ih )+ self.b_ih # Dense layer와 같은 역할함.
        # gate_size가 앞서서 3* hidden_size로 정의되어 있었고 아래 chunk에서 split하는 것 같다.
        gh = tf.matmul(hidden, self.w_hh) + self.b_hh
        i_r, i_i, i_n = tf.split(gi,3,axis = 2)
        h_r, h_i, h_n = tf.split(gh,3,axis = 2)
        resetgate = tf.nn.sigmoid(i_r + h_r)
        inputgate = tf.nn.sigmoid(i_i + h_i)
        newgate = tf.nn.tanh(i_n + resetgate * h_n)
        hy = newgate + inputgate * (hidden - newgate)
        return hy

    def call(self, A, hidden):
        for i in range(self.step):
            hidden = self.Cell(A, hidden)
        return hidden

In [185]:

class TAGNN(tf.keras.models.Model):
    def __init__(self, opt, n_node):
        super(TAGNN, self).__init__()
        self.hidden_size = opt.hiddenSize
        self.n_node = n_node
        self.batch_size = opt.batchSize
        self.nonhybrid = opt.nonhybrid
        # self.embedding = nn.Embedding(self.n_node, self.hidden_size)
        self.embedding = tf.keras.layers.Embedding(input_dim = self.n_node,output_dim=self.hidden_size)
        self.stdv = 1/math.sqrt(self.hidden_size)
        
        self.gnn = GNN(self.hidden_size, step=opt.step)

        # self.linear_one = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_one = tf.keras.layers.Dense(self.hidden_size, use_bias=True,
                                                kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                                bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        # self.linear_two = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_two = tf.keras.layers.Dense(self.hidden_size, use_bias=True,
                                                kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                                bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        # self.linear_three = nn.Linear(self.hidden_size, 1, bias=False)
        self.linear_three = tf.keras.layers.Dense(1 , use_bias=False,
                                                  kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                                  )
        # self.linear_transform = nn.Linear(self.hidden_size * 2, self.hidden_size, bias=True)
        
        self.linear_transform = tf.keras.layers.Dense(self.hidden_size, use_bias=True,
                                                      kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                                      bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))

        # self.linear_t = nn.Linear(self.hidden_size, self.hidden_size, bias=False)  #target attention
        self.linear_t = tf.keras.layers.Dense( self.hidden_size, use_bias=False,
                                              kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                              )
    def call(self, inputs, A,alias_inputs):
        if len(self.embedding.weights) != 0:
          norm = tf.norm(self.embedding.weights[0],axis=1) # 왜 여기도 하는지는 모르겠음.
          self.embedding.weights[0]=self.embedding.weights[0] / tf.reshape(norm,[-1,1])
        hidden = self.embedding(inputs) 
        # inputs = items -> batch_size, len_seq_in_batch
        #print('NAN IN hidden')
        #print(tf.reduce_sum(hidden))


        hidden = self.gnn(A, hidden)
        #print('hidden after gnn')
        #print(hidden)
        
        def _get(j): # index로 slice해서 gather를 사용했음. 
          return tf.gather(hidden[j],alias_inputs[j])
        
        get = lambda j : tf.gather(hidden[j],alias_inputs[j])

        mask =tf.cast(tf.convert_to_tensor(alias_inputs) != 0,'float32')
        mask_new=mask[...,tf.newaxis]

        mask=tf.cast(tf.reduce_sum(mask,axis =-1)!=0,'float32')


        


        #hidden= tf.reduce_sum(tf.stack([get(j) for j in range(len(alias_inputs))]) *mask_new,axis = 2)   # 이 줄에서 아마 retracing 일어나는듯 ? 이 부분에 대한 save에러도 있었음.
        hidden=tf.reduce_sum(tf.map_fn(_get,tf.range(0,alias_inputs.shape[0]), fn_output_signature = tf.float32)*mask_new,axis=2 ) # input과 dtype이 다르다면 반드시 fn_output_signature를 지정해주어야 한다.


        #hidden = tf.stack([get(j) for j in range(len(alias_inputs))])
        #hidden= tf.reduce_sum(tf.stack([get(j) for j in range(len(alias_inputs))]) *mask_new,axis = 2)   # 이 줄에서 아마 retracing 일어나는듯 ? 이 부분에 대한 save에러도 있었음.
        # https://stackoverflow.com/questions/62068323/iterating-over-tf-tensor-is-not-allowed-autograph-is-disabled-in-this-function
        # list comprehension을 제공하지 않는다.

        #tf.reduce_sum(tf.stack([get(j) for j in range(len(alias_inputs))]) *mask_new,axis = 2)

        # batch,max_len,one_day_max,hidden

        #tf.zero(alias_inputs.shape[1],alias_inputs.shape[2],self.hidden_size)

        #print(tf.map_fn(get_, hidden, dtype=tf.float32).shape)


        #print('tf.reduce_sum(tf.math.is_nan(hidden)) after tf.reduce......')
        #print(tf.reduce_sum(tf.cast(tf.math.is_nan(hidden),'float32')))

        hidden = tf.math.divide_no_nan(hidden ,tf.reduce_sum(mask_new,axis =2))
        #print('tf.reduce_sum(tf.math.is_nan(hidden)) after divide...')
        #print(tf.reduce_sum(tf.cast(tf.math.is_nan(hidden),'float32')))
        seq_shape = hidden.shape.as_list() # 여기 해보아야 알듯 리스트로 차원반환하는 것 같은데
        
        # seq_hidden = seq_hidden.view(-1, model.hidden_size) # tf.reshape


        # 아마 이쯤부터 결측 생성되는 것 같음.-
        hidden = tf.reshape(hidden,[-1,self.hidden_size])
        # norms = torch.norm(seq_hidden, p=2, dim=1)  # tf.norm
        norms = tf.norm(hidden,axis =1)
        #print('tf.reduce_sum(tf.math.is_nan(norms))')
        #print(tf.reduce_sum(tf.cast(tf.math.is_nan(norms),'float32')))
        # seq_hidden = seq_hidden.div(norms.unsqueeze(-1).expand_as(seq_hidden)) # unsqueeze에 대해 알아보기
        
        #hidden = hidden / tf.reshape(norms,[-1,1])
        #print('tf.reduce_sum(tf.math.is_nan(hidden)) after hidden/...')
        #print(tf.reduce_sum(tf.cast(tf.math.is_nan(norms),'float32')))
        # expand_as는 broadcasting으로 해결될 수 있는지 확인해보기.
        # seq_hidden = seq_hidden.view(seq_shape) #  
        
        hidden = tf.reshape(hidden,[-1,seq_shape[1],seq_shape[2]])

        #print(self.embedding.weights)


        ht = tf.gather_nd(hidden ,indices = tf.stack([tf.range(hidden.shape[0],dtype='int64') , tf.cast(tf.math.reduce_sum(mask,1),'int64') -1],axis=1))
        # tf.reshape을 이용해야함.
        #print('NAN IN hidden')
        #print(tf.reduce_sum(hidden))
        q1 = tf.reshape(self.linear_one(ht) ,[ht.shape[0], 1, ht.shape[1]])  # batch_size x 1 x latent_size
        #print('q1')
        #print(q1)
        q2 = self.linear_two(hidden)  # batch_size x seq_length x latent_size # 여기가 결측의 시작인듯.
        #print('q2')
        #print(q2)
        alpha = self.linear_three(tf.nn.sigmoid(q1 + q2))  # (b,s,1)
        
        #print('alpha')
        #print(alpha)
        alpha = tf.nn.softmax(alpha, 1) # B,S,1
        #print('alpha')
        #print(alpha)
        #print("ht") # 이거 문제 없음
        #print(ht)
        a = tf.math.reduce_sum(alpha * hidden * tf.cast(tf.reshape(mask, [mask.shape[0], -1, 1]),dtype = 'float32'), 1)  # (b,d)
        #print('a') # nan
        #print(a)
        #print('\n')
        if not self.nonhybrid: # 여기 적용되는지 판단할 것
        
            a = self.linear_transform(tf.concat([a, ht], 1)) 
        
        norm = tf.norm(self.embedding.weights[0],axis=1)
        self.embedding.weights[0]=self.embedding.weights[0] / tf.reshape(norm,[-1,1])

        b = self.embedding.weights[0][1:] # 이 차원이 맞긴 함.

        hidden = hidden * tf.cast(tf.reshape(mask,[mask.shape[0], -1, 1]),'float32')  # batch_size x seq_length x latent_size
        
        #print('hiddne')

        qt = self.linear_t(hidden)  # batch_size x seq_length x latent_size

        beta = tf.nn.softmax(b @ tf.transpose(qt,[0,2,1]), -1)  # batch_size x n_nodes x seq_length
        target = beta @ hidden  # batch_size x n_nodes x latent_size
        a = tf.reshape(a,[ht.shape[0], 1, ht.shape[1]])  # b,1,d
        a = a + target  # b,n,d # 이거 concat인가 ? 

        scores = tf.math.reduce_sum(a * b, -1)  # b,n
        scores=tf.nn.softmax(scores,-1)

        return scores
    def get_config(self):
        return {"hidden_units": self.hidden_size}

    @classmethod
    def from_config(cls, config):
        return cls(**config)

# NISER 논문에서 나온 것처럼 처음 모델이 gnn이전에 embedding layer의 weights을 normalize하는 것을 call 안에서 짜려고 했는데, 처음 embedding layer가 호출 되기 이전까지 weight이 존재하지 않아서 다른 방식으로 짜야할 것 같음.

- build를 따로 둔다거나?
- Input을 사용해서 정의하는 것 ?
sequential이나
funtional api에서 하는 것처럼 일반적인 keras
built in을 사용하면 된다 ?


- 일단 미봉책으로 weight이 길이가 0인 경우 pass 한 다음 이후 embedding layer의 weights이 생겨난 후부터 normalize를 적용하도록 했음.



```python
import numpy as np

inputs = keras.Input(shape=(3,))
outputs = ActivityRegularizationLayer()(inputs)
model = keras.Model(inputs, outputs)

# If there is a loss passed in `compile`, the regularization
# losses get added to it
model.compile(optimizer="adam", loss="mse")
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))

# It's also possible not to pass any loss in `compile`,
# since the model already has a loss to minimize, via the `add_loss`
# call during the forward pass!
model.compile(optimizer="adam")
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))
```

In [186]:
model=TAGNN(oopt,n_node)

loss_ftn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate = oopt.lr)


@tf.function
def train_ftn(item_input,A_input,alias_inputs_,targets_input):
    with tf.GradientTape() as tape :
      scores = model(item_input, A_input,alias_inputs_,training =True)
      #print(scores.numpy()) # 이거 nan 나옴.
      #print(targets_input)
      ll = loss_ftn(targets_input-1,scores)

    
    
    gradients =tape.gradient(ll,model.trainable_weights)

    optimizer.apply_gradients(zip(gradients,model.trainable_weights))
    return ll

In [187]:
@tf.function
def test_ftn(item_input,A_input,alias_inputs_,targets_input):
    scores = model(item_input, A_input, alias_inputs_,training =False)
    scores = tf.math.top_k(scores,k=12).indices
    scores = tf.cast(scores,tf.int64)
    targets_input = tf.cast(targets_input,tf.int64)

    #print(scores == targets_input[:,tf.newaxis]-1)
    result=tf.cast(scores == (targets_input[:,tf.newaxis]-1),tf.float32)

    result_hit = tf.math.reduce_sum(result,axis =1)
    
    result_hit = tf.cast(result_hit != 0,tf.float32) # hit

    index=tf.cast(tf.range(result.shape[-1])+1,tf.float32)


    result_mrr = tf.reduce_sum(result/tf.reshape(index,[-1,index.shape[0]]), axis =-1)


    return result_hit, result_mrr

In [188]:
for epoch in range(oopt.epoch):
    print('-------------------------------------------------------')
    print('epoch: ', epoch)

    slices = train_data1.generate_batch(model.batch_size)
    loss = 0
    for i, j in zip(slices, np.arange(len(slices))):
        alias_inputs, A, items, targets = train_data1.get_slice(i)
        
        alias_inputs = tf.convert_to_tensor(alias_inputs)
        items = tf.convert_to_tensor(items)
        
        A = tf.convert_to_tensor(A)
        
        targets = tf.convert_to_tensor(targets)
        
        l = train_ftn(items,A,alias_inputs,targets)
        
        loss += l 
        
        if j % int(100) == 0:
            print('[%d/%d] Loss: %.4f' % (j, len(slices), l))
        
    
    
    print('\tLoss:\t%.3f' % loss)
    hit, mrr = tf.constant([],dtype= tf.float32), tf.constant([],dtype= tf.float32)
    hhit=[]
    mmrr=[]
    slices = test_data1.generate_batch(model.batch_size)
    for i in slices:
        alias_inputs, A, items, targets = test_data1.get_slice(i)
        
        alias_inputs = tf.convert_to_tensor(alias_inputs)
        items = tf.convert_to_tensor(items)
        
        A = tf.convert_to_tensor(A)
        
        targets = tf.convert_to_tensor(targets)

        h,m=test_ftn(items,A,alias_inputs,targets)

        hit = tf.concat([hit,h],0)
        mrr = tf.concat([mrr,m],0)
        '''
        scores = model(items, A, mask,alias_inputs,training =False)
        scores = tf.math.top_k(scores,k=20).indices
        scores = scores.numpy()
        
            # sub_scores를 tensor에서 numpy로 바꾸고 아래는 그대로 실행하면 됨.
        for score, target, mask in zip(scores, targets, test_data1.mask):
            hhit.append(np.isin(target - 1, score)) # 현재 target은 s_{n+1}인 item이므로, 
                # score 즉, 위에서 추출한 index가 일치하면 hit이라는 list에 append 해준다.

                # score vector에서 target-1과 일치하는 경우를 찾고, 없는 경우 0을 mrr이라는 list에 기록하고,
            if len(np.where(score == target - 1)[0]) == 0:
                mmrr.append(0)
            else:
                # 있는 경우에는 (1/그 위치의 index +1)을 기록한다.
                mmrr.append(1 / (np.where(score == target - 1)[0][0] + 1))
        '''

    hit = np.mean(hit) * 100
    mrr = np.mean(mrr) * 100


    print(f"Recall@12 : {hit} , MRR@12 {mrr}")


-------------------------------------------------------
epoch:  0
[0/198140] Loss: 10.6885
[100/198140] Loss: 10.7026
[200/198140] Loss: 10.3085
[300/198140] Loss: 10.3244
[400/198140] Loss: 10.3200
[500/198140] Loss: 10.0283
[600/198140] Loss: 10.4715
[700/198140] Loss: 10.0880
[800/198140] Loss: 9.7910
[900/198140] Loss: 9.8641
[1000/198140] Loss: 10.3045
[1100/198140] Loss: 9.6231
[1200/198140] Loss: 10.0763
[1300/198140] Loss: 9.4903
[1400/198140] Loss: 11.0940
[1500/198140] Loss: 10.8475
[1600/198140] Loss: 9.9036
[1700/198140] Loss: 9.4160
[1800/198140] Loss: 10.2521
[1900/198140] Loss: 10.2454
[2000/198140] Loss: 10.1328
[2100/198140] Loss: 10.2841
[2200/198140] Loss: 10.0164
[2300/198140] Loss: 10.4383
[2400/198140] Loss: 10.3065
[2500/198140] Loss: 10.0739
[2600/198140] Loss: 10.1255
[2700/198140] Loss: 9.6939
[2800/198140] Loss: 10.3747
[2900/198140] Loss: 10.3827
[3000/198140] Loss: 9.6534
[3100/198140] Loss: 9.6303
[3200/198140] Loss: 9.7447
[3300/198140] Loss: 9.7642
[3400

KeyboardInterrupt: ignored

In [189]:
model.save_weights('./model/0505_niser_200901_256')

In [ ]:
#model.save('./model/0502_niser_200901_256')

```
-------------------------------------------------------
epoch:  0
[0/4402] Loss: 9.1327
WARNING:tensorflow:5 out of the last 6 calls to <function train_ftn at 0x7f80038c4cb0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
WARNING:tensorflow:6 out of the last 7 calls to <function train_ftn at 0x7f80038c4cb0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
[100/4402] Loss: 8.9770
[200/4402] Loss: 8.9180
[300/4402] Loss: 8.4044
[400/4402] Loss: 8.6265
[500/4402] Loss: 8.7719
[600/4402] Loss: 8.4934
[700/4402] Loss: 9.0608
[800/4402] Loss: 8.4043
[900/4402] Loss: 8.3754
[1000/4402] Loss: 9.1329
[1100/4402] Loss: 8.7070
[1200/4402] Loss: 8.5734
[1300/4402] Loss: 8.1753
[1400/4402] Loss: 8.4076
[1500/4402] Loss: 8.9500
[1600/4402] Loss: 8.3972
[1700/4402] Loss: 8.4945
[1800/4402] Loss: 8.4742
[1900/4402] Loss: 9.1580
[2000/4402] Loss: 8.6381
[2100/4402] Loss: 8.2778
[2200/4402] Loss: 8.4068
[2300/4402] Loss: 8.7677
[2400/4402] Loss: 8.1248
[2500/4402] Loss: 8.5282
[2600/4402] Loss: 8.0610
[2700/4402] Loss: 8.3762
[2800/4402] Loss: 8.6859
[2900/4402] Loss: 8.6202
[3000/4402] Loss: 8.6322
[3100/4402] Loss: 8.5665
[3200/4402] Loss: 8.2761
[3300/4402] Loss: 8.9086
[3400/4402] Loss: 8.2455
[3500/4402] Loss: 8.8695
[3600/4402] Loss: 8.3441
[3700/4402] Loss: 8.5327
[3800/4402] Loss: 8.3098
[3900/4402] Loss: 7.8990
[4000/4402] Loss: 8.0932
[4100/4402] Loss: 8.0460
[4200/4402] Loss: 8.4132
[4300/4402] Loss: 8.2627
[4400/4402] Loss: 8.4437
	Loss:	37235.625
Recall@20 : 4.538934305310249 , MRR@20 1.084378082305193
-------------------------------------------------------
epoch:  1
[0/4402] Loss: 8.4622
[100/4402] Loss: 8.6117
[200/4402] Loss: 7.8981
[300/4402] Loss: 7.8315
[400/4402] Loss: 8.1448
[500/4402] Loss: 8.5316
[600/4402] Loss: 7.9694
[700/4402] Loss: 8.6572
[800/4402] Loss: 8.2223
[900/4402] Loss: 8.0679
[1000/4402] Loss: 8.7123
[1100/4402] Loss: 8.0266
[1200/4402] Loss: 8.2721
[1300/4402] Loss: 7.8984
[1400/4402] Loss: 8.2625
[1500/4402] Loss: 8.5136
[1600/4402] Loss: 7.8585
[1700/4402] Loss: 8.0175
[1800/4402] Loss: 8.0727
[1900/4402] Loss: 8.6581
[2000/4402] Loss: 8.3162
[2100/4402] Loss: 8.0461
[2200/4402] Loss: 8.0951
[2300/4402] Loss: 8.3961
[2400/4402] Loss: 7.7614
[2500/4402] Loss: 8.1474
[2600/4402] Loss: 7.4720
[2700/4402] Loss: 7.8181
[2800/4402] Loss: 8.3181
[2900/4402] Loss: 8.2480
[3000/4402] Loss: 8.0860
[3100/4402] Loss: 8.2640
[3200/4402] Loss: 7.6628
[3300/4402] Loss: 8.1966
[3400/4402] Loss: 7.6164
[3500/4402] Loss: 8.6833
[3600/4402] Loss: 7.8270
[3700/4402] Loss: 8.4333
[3800/4402] Loss: 8.0896
[3900/4402] Loss: 7.2534
[4000/4402] Loss: 7.7906
[4100/4402] Loss: 7.5586
[4200/4402] Loss: 8.1443
[4300/4402] Loss: 8.0703
[4400/4402] Loss: 7.9771
	Loss:	35420.930
Recall@20 : 6.506147235631943 , MRR@20 2.2530045360326767
-------------------------------------------------------
epoch:  2
[0/4402] Loss: 8.4803
[100/4402] Loss: 8.3959
[200/4402] Loss: 7.4060
[300/4402] Loss: 7.3159
[400/4402] Loss: 7.9019
[500/4402] Loss: 8.1200
[600/4402] Loss: 7.6013
[700/4402] Loss: 8.6809
[800/4402] Loss: 7.8981
[900/4402] Loss: 7.1553
[1000/4402] Loss: 8.0226
[1100/4402] Loss: 6.9836
[1200/4402] Loss: 8.0866
[1300/4402] Loss: 7.7664
[1400/4402] Loss: 8.0761
[1500/4402] Loss: 8.0080
[1600/4402] Loss: 7.3935
[1700/4402] Loss: 7.7209
[1800/4402] Loss: 7.5963
[1900/4402] Loss: 8.2674
[2000/4402] Loss: 7.9365
[2100/4402] Loss: 7.5936
[2200/4402] Loss: 7.7149
[2300/4402] Loss: 8.1713
[2400/4402] Loss: 7.4022
[2500/4402] Loss: 7.5430
[2600/4402] Loss: 6.9785
[2700/4402] Loss: 7.4260
[2800/4402] Loss: 7.6647
[2900/4402] Loss: 7.8535
[3000/4402] Loss: 7.3364
[3100/4402] Loss: 7.8327
[3200/4402] Loss: 7.2999
[3300/4402] Loss: 7.5044
[3400/4402] Loss: 7.0310
[3500/4402] Loss: 8.3287
[3600/4402] Loss: 7.2841
[3700/4402] Loss: 8.2101
[3800/4402] Loss: 7.8499
[3900/4402] Loss: 6.6918
[4000/4402] Loss: 7.5366
[4100/4402] Loss: 6.9652
[4200/4402] Loss: 7.8374
[4300/4402] Loss: 7.8099
[4400/4402] Loss: 7.4924
	Loss:	33645.664
Recall@20 : 7.079917937517166 , MRR@20 2.9857616871595383
-------------------------------------------------------
epoch:  3
[0/4402] Loss: 8.5211
[100/4402] Loss: 8.3228
[200/4402] Loss: 7.1613
[300/4402] Loss: 6.9613
[400/4402] Loss: 7.6526
[500/4402] Loss: 7.7041
[600/4402] Loss: 7.2721
[700/4402] Loss: 7.9722
[800/4402] Loss: 7.4510
[900/4402] Loss: 6.4149
[1000/4402] Loss: 7.6176
[1100/4402] Loss: 6.1612
[1200/4402] Loss: 7.9348
[1300/4402] Loss: 7.5432
[1400/4402] Loss: 7.8400
[1500/4402] Loss: 7.5729
[1600/4402] Loss: 6.8761
[1700/4402] Loss: 7.6372
[1800/4402] Loss: 7.4500
[1900/4402] Loss: 7.8619
[2000/4402] Loss: 7.6442
[2100/4402] Loss: 6.9684
[2200/4402] Loss: 7.3588
[2300/4402] Loss: 7.9117
[2400/4402] Loss: 7.0074
[2500/4402] Loss: 7.1272
[2600/4402] Loss: 6.6058
[2700/4402] Loss: 7.0542
[2800/4402] Loss: 7.2066
[2900/4402] Loss: 7.5580
[3000/4402] Loss: 6.6977
[3100/4402] Loss: 7.4124
[3200/4402] Loss: 6.9918
[3300/4402] Loss: 6.9365
[3400/4402] Loss: 6.4698
[3500/4402] Loss: 8.0780
[3600/4402] Loss: 6.8603
[3700/4402] Loss: 7.9404
[3800/4402] Loss: 7.6608
[3900/4402] Loss: 6.3413
[4000/4402] Loss: 7.2477
[4100/4402] Loss: 6.4833
[4200/4402] Loss: 7.5917
[4300/4402] Loss: 7.3635
[4400/4402] Loss: 6.9842
	Loss:	32015.068
Recall@20 : 6.8954914808273315 , MRR@20 3.0188381671905518
```

```
epoch:  0
[0/551] Loss: 9.1313
WARNING:tensorflow:5 out of the last 6 calls to <function train_ftn at 0x7fde94691320> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
WARNING:tensorflow:6 out of the last 7 calls to <function train_ftn at 0x7fde94691320> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
[100/551] Loss: 8.5320
[200/551] Loss: 8.4573
[300/551] Loss: 8.3493
[400/551] Loss: 8.4255
[500/551] Loss: 8.4161
	Loss:	4682.043
WARNING:tensorflow:5 out of the last 5 calls to <function test_ftn at 0x7fde9a7d6950> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
WARNING:tensorflow:6 out of the last 6 calls to <function test_ftn at 0x7fde9a7d6950> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
Recall@20 : 3.678278625011444 , MRR@20 0.8045623078942299
-------------------------------------------------------
epoch:  1
[0/551] Loss: 8.4360
[100/551] Loss: 8.3471
[200/551] Loss: 8.3401
[300/551] Loss: 8.1966
[400/551] Loss: 8.2209
[500/551] Loss: 8.2677
	Loss:	4583.788
Recall@20 : 3.9036884903907776 , MRR@20 0.9260150603950024
-------------------------------------------------------
epoch:  2
[0/551] Loss: 8.4063
[100/551] Loss: 8.2909
[200/551] Loss: 8.2130
[300/551] Loss: 8.0507
[400/551] Loss: 8.0739
[500/551] Loss: 8.0964
	Loss:	4504.420
Recall@20 : 3.6885246634483337 , MRR@20 0.7741636596620083
-------------------------------------------------------
epoch:  3
[0/551] Loss: 8.3050
[100/551] Loss: 8.2216
[200/551] Loss: 8.0289
[300/551] Loss: 7.9143
[400/551] Loss: 7.9192
[500/551] Loss: 7.9483
	Loss:	4433.559
Recall@20 : 3.6270491778850555 , MRR@20 0.8505591191351414
-------------------------------------------------------
epoch:  4
[0/551] Loss: 8.1300
[100/551] Loss: 8.1082
[200/551] Loss: 7.8838
[300/551] Loss: 7.7846
[400/551] Loss: 7.8147
[500/551] Loss: 7.7884
	Loss:	4363.283
Recall@20 : 4.231557250022888 , MRR@20 1.0650495998561382
-------------------------------------------------------
epoch:  5
[0/551] Loss: 8.0003
[100/551] Loss: 7.9730
[200/551] Loss: 7.7583
[300/551] Loss: 7.6875
[400/551] Loss: 7.6993
[500/551] Loss: 7.6322
	Loss:	4289.802
Recall@20 : 4.651639237999916 , MRR@20 1.255041267722845
-------------------------------------------------------
epoch:  6
[0/551] Loss: 7.8406
[100/551] Loss: 7.8541
[200/551] Loss: 7.6583
[300/551] Loss: 7.5677
[400/551] Loss: 7.5432
[500/551] Loss: 7.4900
	Loss:	4214.937
Recall@20 : 5.010245740413666 , MRR@20 1.4170462265610695
-------------------------------------------------------
epoch:  7
[0/551] Loss: 7.7513
[100/551] Loss: 7.6584
[200/551] Loss: 7.5355
[300/551] Loss: 7.3945
[400/551] Loss: 7.4355
[500/551] Loss: 7.3660
	Loss:	4139.040
Recall@20 : 5.922131240367889 , MRR@20 1.7022568732500076
-------------------------------------------------------
epoch:  8
[0/551] Loss: 7.6751
[100/551] Loss: 7.5035
[200/551] Loss: 7.3307
[300/551] Loss: 7.2344
[400/551] Loss: 7.2386
[500/551] Loss: 7.2018
	Loss:	4056.599
Recall@20 : 6.014344096183777 , MRR@20 1.9783595576882362
-------------------------------------------------------
epoch:  9
[0/551] Loss: 7.5049
[100/551] Loss: 7.3761
[200/551] Loss: 7.1566
[300/551] Loss: 7.1323
[400/551] Loss: 7.0531
[500/551] Loss: 7.0586
	Loss:	3971.185
Recall@20 : 6.106557324528694 , MRR@20 2.1189233288168907
-------------------------------------------------------
epoch:  10
[0/551] Loss: 7.3151
[100/551] Loss: 7.1911
[200/551] Loss: 6.9490
[300/551] Loss: 6.9698
[400/551] Loss: 6.8792
[500/551] Loss: 6.8710
	Loss:	3884.150
Recall@20 : 6.055327877402306 , MRR@20 2.176481857895851
-------------------------------------------------------
epoch:  11
[0/551] Loss: 7.1611
[100/551] Loss: 7.0065
[200/551] Loss: 6.7958
[300/551] Loss: 6.8369
[400/551] Loss: 6.6549
[500/551] Loss: 6.7200
	Loss:	3800.317
Recall@20 : 6.086065620183945 , MRR@20 2.1360553801059723
-------------------------------------------------------
epoch:  12
[0/551] Loss: 7.0522
[100/551] Loss: 6.8564
[200/551] Loss: 6.6179
[300/551] Loss: 6.6923
[400/551] Loss: 6.5266
[500/551] Loss: 6.6744
	Loss:	3720.019
Recall@20 : 6.045081838965416 , MRR@20 2.235601842403412
-------------------------------------------------------
epoch:  13
[0/551] Loss: 6.9531
[100/551] Loss: 6.7291
[200/551] Loss: 6.4314
[300/551] Loss: 6.5536
[400/551] Loss: 6.3977
[500/551] Loss: 6.5429
	Loss:	3639.909
Recall@20 : 6.116803362965584 , MRR@20 2.2891057655215263
-------------------------------------------------------
epoch:  14
[0/551] Loss: 6.8361
[100/551] Loss: 6.5514
[200/551] Loss: 6.2841

```

In [190]:
import pickle

In [ ]:
open('HandM/0501for_submission_tes.txt', 'rb')

In [ ]:
sub2=pickle.load(open('./HandM/0505for_submission_tes.txt', 'rb'))

In [193]:
sub1=pickle.load(open('HandM/0506for_submission_tra.txt', 'rb'))
item_dict=pickle.load(open('HandM/0506item_dict.txt', 'rb'))

In [194]:
len(sub1[0])

514472

In [197]:
train_sub=sub_Data(sub1,one_day_max=79)
slices = train_sub.generate_batch(model.batch_size)

In [ ]:
out = []
for i, j in zip(slices, np.arange(len(slices))):
    alias_inputs, A, items = train_sub.preprocess(i)
    
    alias_inputs = tf.convert_to_tensor(alias_inputs)
    items = tf.convert_to_tensor(items)
    A = tf.convert_to_tensor(A)
    targets = tf.convert_to_tensor(targets)
    scores = model(items, A, alias_inputs,training =False)
    scores = tf.math.top_k(scores,k=12).indices
    scores = tf.cast(scores,tf.int64) # # 메모리 터짐
    out.append(scores)
    if j % 100 ==0 : print((j / len(slices))*100)

0.0
0.621967906456027
1.243935812912054
1.8659037193680807
2.487871625824108
3.109839532280134
3.7318074387361615
4.353775345192188
4.975743251648216
5.597711158104242
6.219679064560268
6.841646971016295
7.463614877472323
8.08558278392835
8.707550690384377
9.329518596840403
9.951486503296431
10.573454409752458
11.195422316208484
11.81739022266451
12.439358129120537
13.061326035576565


In [122]:
out=tf.concat(out,axis=0)
out=out+1

out_dict={j:i for i,j in item_dict.items()}

out_mapped=np.vectorize(out_dict.get)(out)

out1=pd.DataFrame(out_mapped)
out1="0"+out1

out2=pd.DataFrame(out1.to_string(header=False,
                  index=False,
                  index_names=False).split('\n'))

In [135]:
out2['customer_id'] = sub1[0]
out2.columns = ["out",'customer_id']

In [138]:
submission=pd.read_csv('./HandM/submission.csv')

In [139]:
submission.loc[submission.customer_id.isin(out2.customer_id) ,'prediction'] = out2['out'].values

In [141]:
submission.to_csv('HandM/0505_niser_submission.csv',index=False)

In [ ]:
!pip install kaggle

In [142]:
!kaggle competitions submit -c h-and-m-personalized-fashion-recommendations -f ./HandM/0505_niser_submission.csv -m "Message"

100% 258M/258M [00:21<00:00, 12.6MB/s]
Successfully submitted to H&M Personalized Fashion Recommendations

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 67 bytes


In [ ]:
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [195]:
class sub_Data():
    def __init__(self, data, one_day_max=100,shuffle=False, graph=None):
        inputs = data[1]
        len_max = max_len_check(inputs)
        self.inputs = inputs
        
        self.len_max = len_max
        self.length = len(inputs)
        self.shuffle = shuffle
        self.graph = graph
        self.one_day_max = one_day_max

    def generate_batch(self, batch_size):
        if self.shuffle:
            shuffled_arg = np.arange(self.length)
            np.random.shuffle(shuffled_arg)
            self.inputs = self.inputs[shuffled_arg]
            #self.mask = self.mask[shuffled_arg]
            self.targets = self.targets[shuffled_arg]
        n_batch = int(self.length / batch_size)
        if self.length % batch_size != 0:
            n_batch += 1
        slices = np.split(np.arange(n_batch * batch_size), n_batch)
        slices[-1] = slices[-1][:(self.length - batch_size * (n_batch - 1))]
        return slices
        
    def preprocess(self,i):
        #inputs, mask, targets = self.inputs[i], self.mask[i], self.targets[i]
        inputs = []
        for jk in i :
          inputs.append(self.inputs[jk])

        us_lens = [len(upois) for upois in inputs] # 이게 오래 걸리는듯.
        
        
        inputs = [[j + [0] * (self.one_day_max - len(j)) for j in one_seq] for one_seq in inputs]
        
        
        for idx,le in enumerate(us_lens):
          for jj in range(self.len_max - le):
            inputs[idx].append(self.one_day_max * [0])
        

        
        
        items, n_node, A, alias_inputs = [], [], [], []
        for u_input in inputs:
            n_node.append(len(np.unique(u_input))) 
        max_n_node = np.max(n_node) # 해당 slices 중 한 sequence 안에서 가질 수 있는 아이템의 종류의 최댓값
        # 0 패딩 후에 개수를 따지는 것이기 때문에 id 개수 +1 임.
        
        
        
        for u_input in inputs:
            node = np.unique(u_input)
            
           
            items.append(node.tolist() + (max_n_node - len(node)) * [0])
            
            u_A = np.zeros((max_n_node, max_n_node)) # 균일한 크기로 생성.
            
            
        

            for i in range(len(u_input)):
              # same time point loop
              length=len(u_input[i])
              for j in range(length):
                for k in range(length):
                  if u_input[i][k] == 0 or  u_input[i][j] == 0:
                    break
                  u = np.where(node == u_input[i][j])[0][0]
                  v = np.where(node == u_input[i][k])[0][0]
                  u_A[u][v] +=2
            
            for i in range(len(u_input)):
              for kk in range(len(u_input[i])-1):
                jj=u_input[i][kk]
                ll=u_input[i][kk+1]
                if jj == 0 or ll == 0:
                  break
                u = np.where(node == jj)[0][0]
                v = np.where(node == ll)[0][0]
                u_A[u][v] +=1
              
            
        
            u_sum_in = np.sum(u_A, 0) # degree scaling
            u_sum_in[np.where(u_sum_in == 0)] = 1
            u_A_in = np.divide(u_A, u_sum_in)
            u_sum_out = np.sum(u_A, 1)
            u_sum_out[np.where(u_sum_out == 0)] = 1
            u_A_out = np.divide(u_A.transpose(), u_sum_out)
            u_A = np.concatenate([u_A_in, u_A_out]).transpose()
            A.append(u_A)
            u_alias = []
            for aday in u_input:
              seq_aday = []
              for element in aday:
                seq_aday.append(np.where(node == element)[0][0])
              u_alias.append(seq_aday)
            alias_inputs.append(u_alias)

        return alias_inputs, A, items

# data set을 늘리는 방안도 고려해야함.

### padding part를 get_slices 내부에 넣었음. -> 매 epoch 마다 했던 padding 연산을 다시함. -> 메모리 문제는 더 이상 없어졌으나 굉장히 소모적으로 변함.

### 이에 disk 용량을 사용해서 즉, 미리 padding된 객체를 생성해서 disk에 넣고 꺼내 쓰는 방식을 사용하고자 함.

# 0501 

- 데이터 획득시점 : 2020-09-01
- test set 설정 방법 : 한 sequence를 기준으로 첫 시점이 09-15인 경우를 test set으로 구분.
- duplicates -> drop 했었음.

```

Recall@20 : 0.17144919838756323 , MRR@20 0.02397021889919415
-------------------------------------------------------
epoch:  1
[0/1396] Loss: 10.7619
[100/1396] Loss: 8.1350
[200/1396] Loss: 6.0566
[300/1396] Loss: 7.1981
[400/1396] Loss: 7.2146
[500/1396] Loss: 5.8306
[600/1396] Loss: 5.2453
[700/1396] Loss: 3.4625
[800/1396] Loss: 2.1029
[900/1396] Loss: 1.6762
[1000/1396] Loss: 1.8171
[1100/1396] Loss: 0.0002
[1200/1396] Loss: 0.0019
[1300/1396] Loss: 0.0001
	Loss:	5437.556
Recall@20 : 0.41508753784000874 , MRR@20 0.07311432273127139
-------------------------------------------------------
epoch:  2
[0/1396] Loss: 9.3718
[100/1396] Loss: 7.0771
[200/1396] Loss: 6.2432
[300/1396] Loss: 5.6623
[400/1396] Loss: 5.6069
[500/1396] Loss: 4.0980
[600/1396] Loss: 3.0256
[700/1396] Loss: 0.9985
[800/1396] Loss: 1.1898
[900/1396] Loss: 1.1581
[1000/1396] Loss: 0.1014
[1100/1396] Loss: 0.0008
[1200/1396] Loss: 0.0034
[1300/1396] Loss: 0.0003
	Loss:	3991.758
Recall@20 : 0.7309149950742722 , MRR@20 0.1114008016884327
-------------------------------------------------------
epoch:  3
[0/1396] Loss: 8.3455
[100/1396] Loss: 5.6436
[200/1396] Loss: 4.2199
[300/1396] Loss: 4.1691
[400/1396] Loss: 2.5864
[500/1396] Loss: 1.6439
[600/1396] Loss: 0.3421
[700/1396] Loss: 0.3205
[800/1396] Loss: 0.1477
[900/1396] Loss: 0.1285
[1000/1396] Loss: 0.0224
[1100/1396] Loss: 0.0011
[1200/1396] Loss: 0.0010
[1300/1396] Loss: 0.0001
	Loss:	2508.844
Recall@20 : 2.0573902875185013 , MRR@20 0.5278557073324919
-------------------------------------------------------
epoch:  4
[0/1396] Loss: 8.9669
[100/1396] Loss: 4.1573
[200/1396] Loss: 2.3965
[300/1396] Loss: 1.6168
[400/1396] Loss: 0.8023
[500/1396] Loss: 0.1551
[600/1396] Loss: 0.0367
[700/1396] Loss: 0.0039
[800/1396] Loss: 0.1776
[900/1396] Loss: 0.0233
[1000/1396] Loss: 0.0059
[1100/1396] Loss: 0.0002
[1200/1396] Loss: 0.0001
[1300/1396] Loss: 0.0000
	Loss:	1298.375
Recall@20 : 3.73578779399395 , MRR@20 1.3103222474455833
-------------------------------------------------------
epoch:  5
[0/1396] Loss: 7.4700
[100/1396] Loss: 1.1910
[200/1396] Loss: 0.6365
[300/1396] Loss: 0.6646
[400/1396] Loss: 0.3910
[500/1396] Loss: 1.1450
[600/1396] Loss: 0.0403
[700/1396] Loss: 0.0214
[800/1396] Loss: 0.1528
[900/1396] Loss: 0.0063
[1000/1396] Loss: 0.0035
[1100/1396] Loss: 0.0000
[1200/1396] Loss: 0.0000
[1300/1396] Loss: 0.0000
	Loss:	731.314
Recall@20 : 4.9720268696546555 , MRR@20 2.0377062261104584
-------------------------------------------------------
epoch:  6
[0/1396] Loss: 5.9955
[100/1396] Loss: 1.5992
[200/1396] Loss: 0.4148
[300/1396] Loss: 0.7427
[400/1396] Loss: 0.5668
[500/1396] Loss: 0.3221
[600/1396] Loss: 0.0057
[700/1396] Loss: 0.0009
[800/1396] Loss: 0.1897
[900/1396] Loss: 0.0274
[1000/1396] Loss: 0.0020
[1100/1396] Loss: 0.0001
[1200/1396] Loss: 0.0007
[1300/1396] Loss: 0.0001
	Loss:	649.514
Recall@20 : 5.161523073911667 , MRR@20 2.151380106806755
-------------------------------------------------------
epoch:  7
[0/1396] Loss: 4.4476
[100/1396] Loss: 1.6693
[200/1396] Loss: 1.6100
[300/1396] Loss: 0.9982
[400/1396] Loss: 0.9884
[500/1396] Loss: 0.2785
[600/1396] Loss: 0.2758
[700/1396] Loss: 0.0462
[800/1396] Loss: 0.2909
[900/1396] Loss: 0.0534
[1000/1396] Loss: 0.0020
[1100/1396] Loss: 0.0001
[1200/1396] Loss: 0.0008
[1300/1396] Loss: 0.0000
	Loss:	705.316
Recall@20 : 6.64140060544014 , MRR@20 3.0510060489177704
-------------------------------------------------------
epoch:  8
[0/1396] Loss: 5.4679
[100/1396] Loss: 1.2573
[200/1396] Loss: 0.7012
[300/1396] Loss: 2.2229
[400/1396] Loss: 1.1801
[500/1396] Loss: 0.1892
[600/1396] Loss: 0.0034
[700/1396] Loss: 0.0001
[800/1396] Loss: 0.1617
[900/1396] Loss: 0.0008
[1000/1396] Loss: 0.0018
[1100/1396] Loss: 0.0000
[1200/1396] Loss: 0.0000
[1300/1396] Loss: 0.0000
	Loss:	612.357
Recall@20 : 6.876014918088913 , MRR@20 3.1907230615615845
-------------------------------------------------------
epoch:  9
[0/1396] Loss: 3.2161
[100/1396] Loss: 0.9919
[200/1396] Loss: 0.4371
[300/1396] Loss: 1.0255
[400/1396] Loss: 0.5027
[500/1396] Loss: 0.8706
[600/1396] Loss: 0.0030
[700/1396] Loss: 0.0007
[800/1396] Loss: 0.1721
[900/1396] Loss: 0.3536
[1000/1396] Loss: 0.0005
[1100/1396] Loss: 0.0001
[1200/1396] Loss: 0.0000
[1300/1396] Loss: 0.0000
	Loss:	583.171
Recall@20 : 7.444504648447037 , MRR@20 3.347522020339966
-------------------------------------------------------
epoch:  10
[0/1396] Loss: 3.3454
[100/1396] Loss: 1.1065
[200/1396] Loss: 0.4173
[300/1396] Loss: 0.9339
[400/1396] Loss: 0.2872
[500/1396] Loss: 0.1755
[600/1396] Loss: 0.0104
[700/1396] Loss: 0.0109
[800/1396] Loss: 0.2029
[900/1396] Loss: 0.0005
[1000/1396] Loss: 0.0008
[1100/1396] Loss: 0.0000
[1200/1396] Loss: 0.0002
[1300/1396] Loss: 0.0000
	Loss:	554.232
Recall@20 : 7.579859346151352 , MRR@20 3.700995445251465
-------------------------------------------------------
epoch:  11
[0/1396] Loss: 3.1046
[100/1396] Loss: 1.4957
[200/1396] Loss: 0.3188
[300/1396] Loss: 0.6736
[400/1396] Loss: 0.2668
[500/1396] Loss: 0.1906
[600/1396] Loss: 0.0987
[700/1396] Loss: 0.0018
[800/1396] Loss: 0.3331
[900/1396] Loss: 0.0033
[1000/1396] Loss: 0.0206
[1100/1396] Loss: 0.0001
[1200/1396] Loss: 0.0001
[1300/1396] Loss: 0.0000
	Loss:	539.397
Recall@20 : 7.11965337395668 , MRR@20 3.4687120467424393
-------------------------------------------------------
epoch:  12
[0/1396] Loss: 2.8918
[100/1396] Loss: 1.6235
[200/1396] Loss: 0.3028
[300/1396] Loss: 1.7735
[400/1396] Loss: 0.5591
[500/1396] Loss: 0.2507
[600/1396] Loss: 0.0001
[700/1396] Loss: 0.0052
[800/1396] Loss: 0.2073
[900/1396] Loss: 0.0053
[1000/1396] Loss: 0.0026
[1100/1396] Loss: 0.0000
[1200/1396] Loss: 0.0000
[1300/1396] Loss: 0.0000
	Loss:	506.588
Recall@20 : 7.236960530281067 , MRR@20 3.569217026233673
-------------------------------------------------------
epoch:  13
[0/1396] Loss: 2.9235
[100/1396] Loss: 0.6941
[200/1396] Loss: 0.5572
[300/1396] Loss: 1.0340
[400/1396] Loss: 0.4004
[500/1396] Loss: 0.4084
[600/1396] Loss: 0.0006
[700/1396] Loss: 0.0079
[800/1396] Loss: 0.1796
[900/1396] Loss: 0.1135
[1000/1396] Loss: 0.0009
[1100/1396] Loss: 0.0000
[1200/1396] Loss: 0.0000
[1300/1396] Loss: 0.0000
	Loss:	493.460
Recall@20 : 7.209889590740204 , MRR@20 3.4221820533275604
-------------------------------------------------------
epoch:  14
[0/1396] Loss: 3.4950
[100/1396] Loss: 0.9645
[200/1396] Loss: 0.3847
[300/1396] Loss: 0.7333
[400/1396] Loss: 0.4559
[500/1396] Loss: 0.8811
[600/1396] Loss: 0.0075
[700/1396] Loss: 0.0000
[800/1396] Loss: 0.1666
[900/1396] Loss: 0.0002
[1000/1396] Loss: 0.0003
[1100/1396] Loss: 0.0001
[1200/1396] Loss: 0.0001
[1300/1396] Loss: 0.0000
	Loss:	482.189
Recall@20 : 7.2550080716609955 , MRR@20 3.655760735273361
-------------------------------------------------------
epoch:  15
[0/1396] Loss: 2.9218
[100/1396] Loss: 0.7126
[200/1396] Loss: 0.3196
[300/1396] Loss: 0.5116
[400/1396] Loss: 0.5754
[500/1396] Loss: 0.1065
[600/1396] Loss: 0.0001
[700/1396] Loss: 0.0033
[800/1396] Loss: 0.1723
[900/1396] Loss: 0.0192
[1000/1396] Loss: 0.0007
[1100/1396] Loss: 0.0000
[1200/1396] Loss: 0.0000
[1300/1396] Loss: 0.0000
	Loss:	475.000
Recall@20 : 7.579859346151352 , MRR@20 3.734191507101059
-------------------------------------------------------
epoch:  16
[0/1396] Loss: 3.0302
[100/1396] Loss: 0.6599
[200/1396] Loss: 0.7916
[300/1396] Loss: 0.8719
[400/1396] Loss: 0.1834
[500/1396] Loss: 0.1710
[600/1396] Loss: 0.0151
[700/1396] Loss: 0.0103
[800/1396] Loss: 0.2309
[900/1396] Loss: 0.0020
[1000/1396] Loss: 0.0009
[1100/1396] Loss: 0.0000
[1200/1396] Loss: 0.0001
[1300/1396] Loss: 0.0000
	Loss:	455.962
Recall@20 : 8.15737247467041 , MRR@20 3.932087868452072
-------------------------------------------------------
epoch:  17
[0/1396] Loss: 3.2356
[100/1396] Loss: 0.7728
[200/1396] Loss: 0.4321
[300/1396] Loss: 0.9849
[400/1396] Loss: 0.2544
[500/1396] Loss: 0.2003
[600/1396] Loss: 0.0005
[700/1396] Loss: 0.0006
[800/1396] Loss: 0.4181
[900/1396] Loss: 0.1147
[1000/1396] Loss: 0.0005
[1100/1396] Loss: 0.0000
[1200/1396] Loss: 0.0001
[1300/1396] Loss: 0.0000
	Loss:	441.683
Recall@20 : 7.715214043855667 , MRR@20 3.807774931192398
-------------------------------------------------------
epoch:  18
[0/1396] Loss: 3.3083
[100/1396] Loss: 0.7006
[200/1396] Loss: 0.4273
[300/1396] Loss: 1.0539
[400/1396] Loss: 0.2600
[500/1396] Loss: 0.4774
[600/1396] Loss: 0.0251
[700/1396] Loss: 0.0011
[800/1396] Loss: 0.3265
[900/1396] Loss: 0.0000
[1000/1396] Loss: 0.0006
[1100/1396] Loss: 0.0000
[1200/1396] Loss: 0.0000
[1300/1396] Loss: 0.0000
	Loss:	426.165
Recall@20 : 8.15737247467041 , MRR@20 3.850197419524193
-------------------------------------------------------
epoch:  19
[0/1396] Loss: 2.8038
[100/1396] Loss: 0.6553
[200/1396] Loss: 0.3520
[300/1396] Loss: 0.5705
[400/1396] Loss: 0.4822
[500/1396] Loss: 0.6115
[600/1396] Loss: 0.0010
[700/1396] Loss: 0.0377
[800/1396] Loss: 0.2922
[900/1396] Loss: 0.0085
[1000/1396] Loss: 0.0004
[1100/1396] Loss: 0.0000
[1200/1396] Loss: 0.0000
[1300/1396] Loss: 0.0000
	Loss:	420.555
Recall@20 : 7.462552189826965 , MRR@20 3.706730529665947
-------------------------------------------------------
epoch:  20
[0/1396] Loss: 2.9215
[100/1396] Loss: 1.0358
[200/1396] Loss: 0.3942
[300/1396] Loss: 0.6650
[400/1396] Loss: 0.3501
[500/1396] Loss: 0.1506
[600/1396] Loss: 0.2228
[700/1396] Loss: 0.0003
[800/1396] Loss: 0.1668
[900/1396] Loss: 0.0068
[1000/1396] Loss: 0.0009
[1100/1396] Loss: 0.0000
[1200/1396] Loss: 0.0000
[1300/1396] Loss: 0.0000
	Loss:	417.759
Recall@20 : 7.5979068875312805 , MRR@20 3.7883199751377106
-------------------------------------------------------
epoch:  21
[0/1396] Loss: 2.9510
[100/1396] Loss: 0.6082
[200/1396] Loss: 0.2534
[300/1396] Loss: 0.9363
[400/1396] Loss: 0.3094
[500/1396] Loss: 0.5843
[600/1396] Loss: 0.0002
[700/1396] Loss: 0.0001
[800/1396] Loss: 0.1751
[900/1396] Loss: 0.1381
[1000/1396] Loss: 0.0007
[1100/1396] Loss: 0.0000
[1200/1396] Loss: 0.0000
[1300/1396] Loss: 0.0000
	Loss:	407.660
Recall@20 : 7.128676772117615 , MRR@20 3.7243787199258804
-------------------------------------------------------
epoch:  22
[0/1396] Loss: 2.9615
[100/1396] Loss: 1.0033
[200/1396] Loss: 0.5204
[300/1396] Loss: 0.5787
[400/1396] Loss: 0.4209
[500/1396] Loss: 0.2499
[600/1396] Loss: 0.0001
[700/1396] Loss: 0.0024
[800/1396] Loss: 0.2402
[900/1396] Loss: 0.0000
[1000/1396] Loss: 0.0006
[1100/1396] Loss: 0.0000
[1200/1396] Loss: 0.0000
[1300/1396] Loss: 0.0000
	Loss:	413.572
Recall@20 : 7.426457107067108 , MRR@20 3.6325834691524506
-------------------------------------------------------
epoch:  23
[0/1396] Loss: 2.7388
[100/1396] Loss: 0.6911
[200/1396] Loss: 0.4048
[300/1396] Loss: 0.5865
[400/1396] Loss: 0.3651
[500/1396] Loss: 0.2849
[600/1396] Loss: 0.0006
[700/1396] Loss: 0.0082
[800/1396] Loss: 0.1567
[900/1396] Loss: 0.0012
[1000/1396] Loss: 0.0000
[1100/1396] Loss: 0.0000
[1200/1396] Loss: 0.0000
[1300/1396] Loss: 0.0000
	Loss:	388.499
Recall@20 : 7.688143104314804 , MRR@20 3.889022395014763
-------------------------------------------------------
epoch:  24
[0/1396] Loss: 2.5770
[100/1396] Loss: 0.6097
[200/1396] Loss: 0.2550
[300/1396] Loss: 1.3593
[400/1396] Loss: 0.4429
[500/1396] Loss: 0.1640
[600/1396] Loss: 0.0028
[700/1396] Loss: 0.0016
[800/1396] Loss: 0.1537
[900/1396] Loss: 0.1958
[1000/1396] Loss: 0.0014
[1100/1396] Loss: 0.0000
[1200/1396] Loss: 0.0003
[1300/1396] Loss: 0.0001
	Loss:	392.554
Recall@20 : 7.372315227985382 , MRR@20 3.489958494901657
-------------------------------------------------------
epoch:  25
[0/1396] Loss: 2.9666
[100/1396] Loss: 0.6565
[200/1396] Loss: 1.0836
[300/1396] Loss: 0.7324
[400/1396] Loss: 0.4313
[500/1396] Loss: 0.2031
[600/1396] Loss: 0.0044
[700/1396] Loss: 0.0097
[800/1396] Loss: 0.1731
[900/1396] Loss: 0.0004
[1000/1396] Loss: 0.0010
[1100/1396] Loss: 0.0000
[1200/1396] Loss: 0.0001
[1300/1396] Loss: 0.0000
	Loss:	388.989
Recall@20 : 7.128676772117615 , MRR@20 3.722606599330902
-------------------------------------------------------
epoch:  26
[0/1396] Loss: 2.7439
[100/1396] Loss: 1.1669
[200/1396] Loss: 0.4727
[300/1396] Loss: 0.5670
[400/1396] Loss: 0.2277
[500/1396] Loss: 0.2410
[600/1396] Loss: 0.0001
[700/1396] Loss: 0.0001
[800/1396] Loss: 0.2020
[900/1396] Loss: 0.0005
[1000/1396] Loss: 0.0004
[1100/1396] Loss: 0.0001
[1200/1396] Loss: 0.0000
[1300/1396] Loss: 0.0000
	Loss:	389.218
Recall@20 : 7.4715755879879 , MRR@20 3.7830282002687454
-------------------------------------------------------
epoch:  27
[0/1396] Loss: 2.5396
[100/1396] Loss: 0.6617
[200/1396] Loss: 0.3004
[300/1396] Loss: 0.6330
[400/1396] Loss: 0.1818
[500/1396] Loss: 0.0997
[600/1396] Loss: 0.0096
[700/1396] Loss: 0.0049
[800/1396] Loss: 0.1628
[900/1396] Loss: 0.0228
[1000/1396] Loss: 0.0002
[1100/1396] Loss: 0.0000
[1200/1396] Loss: 0.0001
[1300/1396] Loss: 0.0000
	Loss:	369.752
Recall@20 : 7.137700915336609 , MRR@20 3.5162944346666336
-------------------------------------------------------
epoch:  28
[0/1396] Loss: 2.2986
[100/1396] Loss: 0.7124
[200/1396] Loss: 0.3090
[300/1396] Loss: 0.6825
[400/1396] Loss: 0.2792
[500/1396] Loss: 0.4574
[600/1396] Loss: 0.0020
[700/1396] Loss: 0.0010
[800/1396] Loss: 0.1708
[900/1396] Loss: 0.0009
[1000/1396] Loss: 0.0001
[1100/1396] Loss: 0.0001
[1200/1396] Loss: 0.0000
[1300/1396] Loss: 0.0000
	Loss:	365.582
Recall@20 : 7.291102409362793 , MRR@20 3.6785967648029327
-------------------------------------------------------
epoch:  29
[0/1396] Loss: 2.6138
[100/1396] Loss: 0.9707
[200/1396] Loss: 0.2130
[300/1396] Loss: 0.7378
[400/1396] Loss: 0.6036
[500/1396] Loss: 0.1433
[600/1396] Loss: 0.0004
[700/1396] Loss: 0.0102
[800/1396] Loss: 0.2366
[900/1396] Loss: 0.2245
[1000/1396] Loss: 0.0001
[1100/1396] Loss: 0.0000
[1200/1396] Loss: 0.0001
[1300/1396] Loss: 0.0000
	Loss:	364.168
Recall@20 : 7.426457107067108 , MRR@20 3.912309929728508
```

```
[0/1396] Loss: 9.3718
[100/1396] Loss: 7.0771
[200/1396] Loss: 6.2432
[300/1396] Loss: 5.6623
[400/1396] Loss: 5.6069
[500/1396] Loss: 4.0980
[600/1396] Loss: 3.0256
[700/1396] Loss: 0.9985
[800/1396] Loss: 1.1898
[900/1396] Loss: 1.1581
[1000/1396] Loss: 0.1014
[1100/1396] Loss: 0.0008
[1200/1396] Loss: 0.0034
[1300/1396] Loss: 0.0003
	Loss:	3991.758
Recall@20 : 0.7309149950742722 , MRR@20 0.1114008016884327
```

- 두번째 epoch.
```
[0/1396] Loss: 10.7619
[100/1396] Loss: 8.1350
[200/1396] Loss: 6.0566
[300/1396] Loss: 7.1981
[400/1396] Loss: 7.2146
[500/1396] Loss: 5.8306
[600/1396] Loss: 5.2453
[700/1396] Loss: 3.4625
[800/1396] Loss: 2.1029
[900/1396] Loss: 1.6762
[1000/1396] Loss: 1.8171
[1100/1396] Loss: 0.0002
[1200/1396] Loss: 0.0019
[1300/1396] Loss: 0.0001
	Loss:	5437.55
Recall@20 : 0.41508753784000874 , MRR@20 0.07311432273127139
```

```
0501 첫번째 epoch

[200/1396] Loss: 8.8053
[300/1396] Loss: 8.8630
[400/1396] Loss: 8.9722
[500/1396] Loss: 8.8889
[600/1396] Loss: 8.8052
[700/1396] Loss: 8.5113
[800/1396] Loss: 6.5697
[900/1396] Loss: 7.6722
[1000/1396] Loss: 6.6189
[1100/1396] Loss: 0.0000
[1200/1396] Loss: 0.0019
[1300/1396] Loss: 0.0000

Recall@20 : 0.17144919838756323 , MRR@20 0.02397021889919415
```


In [ ]:
for i, j in zip(slices, np.arange(len(slices))):
    alias_inputs, A, items, targets = train_data1.get_slice(i)

In [ ]:
alias_inputs, A, items, targets = train_data1.get_slice(i)

In [ ]:
targets -1

In [ ]:
for i, j in zip(slices, np.arange(len(slices))):
  pass

In [ ]:
i

Node: 'tagnn_42/embedding_42/embedding_lookup'
indices[79,1] = 9608 is not in [0, 9608)
	 [[{{node tagnn_42/embedding_42/embedding_lookup}}]] [Op:__inference_train_ftn_1000891]


In [ ]:
mrr

In [ ]:
np.mean(hhit)

In [ ]:
mmrr

In [ ]:
    scores = tf.math.top_k(scores,k=20).indices
    scores = tf.cast(scores,tf.int64)
    scores = tf.cast(targets,tf.int64)
    
    result=tf.cast(scores == targets[:,tf.newaxis]-1,tf.float32)

In [ ]:
result

In [ ]:
hit=[]
mrr=[]

In [ ]:
#        scores = model(items, A, mask,alias_inputs,training =False)
scores = tf.math.top_k(scores,k=20).indices
scores = scores.numpy()
for score, target, mask in zip(scores, targets, test_data1.mask):
    hit.append(np.isin(target - 1, score)) # 현재 target은 s_{n+1}인 item이므로, 
        # score 즉, 위에서 추출한 index가 일치하면 hit이라는 list에 append 해준다.

        # score vector에서 target-1과 일치하는 경우를 찾고, 없는 경우 0을 mrr이라는 list에 기록하고,
    if len(np.where(score == target - 1)[0]) == 0:
        mrr.append(0)
    else:
        # 있는 경우에는 (1/그 위치의 index +1)을 기록한다.
        mrr.append(1 / (np.where(score == target - 1)[0][0] + 1))

In [ ]:
hit

# 1회 epoch
```
 Recall@20 : 45.50432957593784 , MRR@20 15.308210672538506
```
# 2회 epoch

```
Recall@20 : 51.77839242775932 , MRR@20 17.690032606197565
```

# 3회 epoch
```
Recall@20 : 52.64569752734653 , MRR@20 18.16359554471299
```

Recall@20 : 51.98409940650757 , MRR@20 18.082734233101373